In [1]:
import pandas as pd

# Load the datasets
facebookData = pd.read_csv("facebookFINAL.csv", quotechar='"', dtype=str, on_bad_lines="warn", engine="python")
googleData = pd.read_csv("google.csv", quotechar='"', dtype=str, on_bad_lines="warn", engine="python")
websiteData = pd.read_csv("website.csv", quotechar='"', dtype=str, on_bad_lines="warn", engine="python")

def resolve_conflicts(row, col_x, col_y):
    if pd.isna(row[col_x]) or row[col_x] == '':
        return row[col_y]
    return row[col_x]

# Adjust chunk size based on memory
chunk_size = 100000  # Adjust based on memory limits

# Initialize an empty DataFrame to hold merged results
merged_results = pd.DataFrame()

# Process in chunks for websiteData
for chunk in pd.read_csv("website.csv", chunksize=chunk_size, quotechar='"', dtype=str, on_bad_lines="warn"):
    # Rename columns in websiteData to match googleData for merging
    chunk.rename(columns={"root_domain": "domain", 
                          "legal_name": "name", 
                          "main_city": "city", 
                          "main_country": "country_name", 
                          "main_region": "region_name", 
                          "s_category": "category"}, inplace=True)

    # Fill missing values with an empty string to avoid NaN issues during merge
    chunk.fillna('', inplace=True)

    # Ensure certain columns are lowercase for consistency
    chunk["name"] = chunk["name"].apply(str.lower)
    chunk["region_name"] = chunk["region_name"].apply(str.lower)
    chunk["country_name"] = chunk["country_name"].apply(str.lower)
    chunk["category"] = chunk["category"].apply(str.lower)

    # Drop unnecessary columns
    chunk.drop(columns=["domain_suffix", "tld"], inplace=True)

    # Merge websiteData chunk with googleData based on common columns
    merged_chunk = pd.merge(chunk, googleData, how='right', on=["phone"], suffixes=('_x', '_y'))

    # Apply conflict resolution for necessary columns after merging
    if not merged_results.empty:
        merged_chunk['domain'] = merged_chunk.apply(lambda row: resolve_conflicts(row, 'domain_x', 'domain_y'), axis=1)
        merged_chunk['name'] = merged_chunk.apply(lambda row: resolve_conflicts(row, 'name_x', 'name_y'), axis=1)
        merged_chunk['city'] = merged_chunk.apply(lambda row: resolve_conflicts(row, 'city_x', 'city_y'), axis=1)
        merged_chunk['country_name'] = merged_chunk.apply(lambda row: resolve_conflicts(row, 'country_name_x', 'country_name_y'), axis=1)
        merged_chunk['region_name'] = merged_chunk.apply(lambda row: resolve_conflicts(row, 'region_name_x', 'region_name_y'), axis=1)
    else:
        # If merged_results is empty, create the new columns directly
        merged_chunk['domain'] = merged_chunk.apply(lambda row: resolve_conflicts(row, 'domain_x', 'domain_y'), axis=1)
        merged_chunk['name'] = merged_chunk.apply(lambda row: resolve_conflicts(row, 'name_x', 'name_y'), axis=1)
        merged_chunk['city'] = merged_chunk.apply(lambda row: resolve_conflicts(row, 'city_x', 'city_y'), axis=1)
        merged_chunk['country_name'] = merged_chunk.apply(lambda row: resolve_conflicts(row, 'country_name_x', 'country_name_y'), axis=1)
        merged_chunk['region_name'] = merged_chunk.apply(lambda row: resolve_conflicts(row, 'region_name_x', 'region_name_y'), axis=1)

    # Concatenate the merged chunk with the merged_results DataFrame
    merged_results = pd.concat([merged_results, merged_chunk], ignore_index=True)

# Drop all columns ending with _x and _y
merged_results = merged_results.loc[:, ~merged_results.columns.str.endswith(('_x', '_y'))]
merged_results=merged_results.drop(columns=["phone_country_code"])


# Save the final merged results to a CSV file
merged_results.to_csv("anomalie.csv", encoding="utf-8-sig", index=False)

# Print final column names and length
print(merged_results.columns)
print(len(merged_results.columns))


Index(['language', 'phone', 'site_name', 'address', 'country_code',
       'raw_address', 'raw_phone', 'region_code', 'text', 'zip_code',
       'number_reviews', 'customer_rating', 'years_experience', 'domain',
       'name', 'city', 'country_name', 'region_name'],
      dtype='object')
18


In [49]:
# import pandas as pd

# # Set chunk size
# chunk_size = 6000  # Adjust based on your memory constraints

# # Initialize an empty DataFrame to hold the final results
# final_results = pd.DataFrame()

# # Process facebookData in chunks
# for facebook_chunk in pd.read_csv("facebookFINAL.csv", chunksize=chunk_size, quotechar='"', dtype=str, on_bad_lines="warn"):
    
#     # Process merged_results in chunks
#     for merged_chunk in pd.read_csv("anomalie.csv", chunksize=chunk_size, quotechar='"', dtype=str, on_bad_lines="warn"):
        
#         # Merge the current chunk of merged_results with the current chunk of facebookData
#         chunk_merged = pd.merge(merged_chunk, facebook_chunk, how='left', on=["phone"], suffixes=('', '_fb'))
        
#         # Resolve conflicts for necessary columns
#         chunk_merged['domain'] = chunk_merged.apply(lambda row: resolve_conflicts(row, 'domain', 'domain_fb'), axis=1)
#         chunk_merged['name'] = chunk_merged.apply(lambda row: resolve_conflicts(row, 'name', 'name_fb'), axis=1)
#         chunk_merged['city'] = chunk_merged.apply(lambda row: resolve_conflicts(row, 'city', 'city_fb'), axis=1)
#         chunk_merged['country_name'] = chunk_merged.apply(lambda row: resolve_conflicts(row, 'country_name', 'country_name_fb'), axis=1)
#         chunk_merged['region_name'] = chunk_merged.apply(lambda row: resolve_conflicts(row, 'region_name', 'region_name_fb'), axis=1)

#         # Drop the columns ending with _fb
#         chunk_merged = chunk_merged.loc[:, ~chunk_merged.columns.str.endswith('_fb')]

#         # Concatenate the chunk to final_results
#         final_results = pd.concat([final_results, chunk_merged], ignore_index=True)

# # Save the final results to a new CSV file
# final_results.to_csv("final_anomalie.csv", encoding="utf-8-sig", index=False)

# # Print final column names and length
# print(final_results.columns)
# print(len(final_results.columns))


MemoryError: Unable to allocate 506. MiB for an array with shape (19, 3491664) and data type object